In [ ]:
import datetime as dt
import sys, os
import dateutil.relativedelta as rd
import json
import pathlib as pl
from typing import List
import warnings

import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
import shapely.geometry as sg
import shapely.ops as so
import folium as fl

DIR = pl.Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk


warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

DATA = DIR / 'data'

In [ ]:
#feed = gk.read_feed(DATA / "cairns_gtfs.zip", dist_units='m').append_dist_to_stop_times()
feed_path = (
    pl.Path.home() 
    / "mrcagney/projects/transit_flows_data"
    / "data/sources/auckland_20250211/auckland_gtfs_20250210.zip"
)
feed = gk.read_feed(feed_path, dist_units="m")
dates = feed.get_first_week()[:1]
route_id = "CTY-202"  # A non-simple looping route

# Improve shape_dist_traveled field
feed2 = feed.restrict_to_routes([route_id]).append_dist_to_stop_times()

trip_id = feed2.get_trips(dates[0]).loc[lambda x: x["route_id"] == route_id, "trip_id"].iat[0]
m = feed2.map_trips([trip_id], show_stops=True, show_direction=True)

gj = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            174.76597006200313,
            -36.84595455878999
          ],
          [
            174.76639610648039,
            -36.846031885888046
          ]
        ],
        "type": "LineString"
      }
    }
  ]
}
screen_line = gpd.GeoDataFrame.from_features(gj).set_crs(gk.WGS84)

# Add screen line to map. Folium, why isn't this easier?!
# Remove existing layer control and fit bounds
keys_to_remove = [
    key for key in m._children.keys() 
    if key.startswith('layer_control_') or key.startswith('fit_bounds_')
]
for key in keys_to_remove:
    m._children.pop(key)

# Create and add the new feature group of screen line
fg = fl.FeatureGroup(name="Screen lines")
fl.GeoJson(
    screen_line,     
    style_function=lambda feature: {
        'color': 'red',
        'weight': 2,
    },
).add_to(fg)
fg.add_to(m)

# Add a new layer control and fit bounds
fl.LayerControl().add_to(m)
m.fit_bounds(fg.get_bounds())

display(m)

# Show screen line counts restricted to trip
slc = feed2.compute_screen_line_counts(screen_line, dates=dates, segmentize_m=1)
slc.loc[lambda x: x["trip_id"] == trip_id]


In [ ]:
display(feed.stop_times.loc[lambda x: x["trip_id"] == trip_id])
feed.restrict_to_routes([route_id]).append_dist_to_stop_times().stop_times.loc[lambda x: x["trip_id"] == trip_id]

In [ ]:
feed.append_dist_to_stop_times?

In [ ]:
feed_path = "http://web.mta.info/developers/data/nyct/subway/google_transit.zip"
feed = gk.read_feed(feed_path, dist_units="mi")

# Grab a stop with a parent station
stop_id = feed.stops.loc[lambda x: x["parent_station"].notna(), "stop_id"].iat[0]
parent_id = feed.stops.loc[lambda x: x["stop_id"] == stop_id, "parent_station"].iat[0]
# Get one of its trips
trip_id = feed.stop_times.loc[lambda x: x["stop_id"] == stop_id, "trip_id"].iat[0]
# Restrict feed to that trip and resulting feed should have that parent station
feed2 = feed.restrict_to_trips([trip_id])
assert parent_id in feed2.stops["stop_id"].values
assert parent_id in feed2.transfers["from_stop_id"].values

In [ ]:
feed.get_shapes(as_gdf=True)

In [ ]:
gk.geometrize_shapes(feed.shapes.iloc[:1])

In [ ]:
ts = feed.compute_trip_stats()
display(ts.head().T)
ts.to_csv(DATA / "cairns_trip_stats.csv", index=False)

In [ ]:
# Maybe a useful function to add
def compute_stop_time_stats(feed):
    if "shape_dist_traveled" in feed.stop_times.columns:
        st = feed.stop_times.copy()
    else:
        st = feed.append_dist_to_stop_times().stop_times
    f = st.sort_values(["trip_id", "stop_sequence"])

    # Choose strategy for calculating duration to next stop
    if (
        f["arrival_time"].count() > 0.5 * f["arrival_time"].shape[0] 
        and f["departure_time"].count() > 0.5 * f["departure_time"].shape[0]
    ):
        # Use stop departure to next stop arrival as duration
        f["atime"] = f["arrival_time"].map(gk.timestr_to_seconds)
        f["dtime"] = f["departure_time"].map(gk.timestr_to_seconds)
        f["duration_to_next_stop"] = f.groupby("trip_id")["atime"].shift(-1) - f["dtime"] 
    elif f["arrival_time"].count() > 0.5 * f["arrival_time"].shape[0]:
        # Use stop arrival to next stop arrival as duration
        f["atime"] = f["arrival_time"].apply(gk.timestr_to_seconds)
        f["duration_to_next_stop"] = f.groupby("trip_id")["atime"].diff().shift(-1) 
    else:
        # Nothing satisfactory, so just use departure times and get many NAs
        f["dtime"] = f["departure_time"].map(gk.timestr_to_seconds)
        f["duration_to_next_stop"] = f.groupby("trip_id")["dtime"].diff().shift(-1) 

    # convert to km or mi
    if gk.is_metric(feed.dist_units):
        g = gk.get_convert_dist(feed.dist_units, "km")
    else:
        g = gk.get_convert_dist(feed.dist_units, "mi")

    f["distance_to_next_stop"] = f.groupby('trip_id')['shape_dist_traveled'].diff().shift(-1)  # feed.dist_units
    f["speed_to_next_stop"] = f["distance_to_next_stop"].map(g) / (f["duration_to_next_stop"] / 3600) # km/h or mi/h
    return f.drop(["atime", "dtime"], axis=1)

compute_stop_time_stats(feed).head(38)
    


In [ ]:
g = feed.geometrize_stops()
p = g["geometry"].iat[0]
type(p)
q = sg.MultiPoint([p, p])
list(q.geoms)

In [ ]:
feed.map_trips(trip_ids[:1], include_stops=True, include_arrows=True)

In [ ]:
dates = feed.get_first_week()[6:]
activity = feed.compute_trip_activity(dates)
display(activity)

trip_ids = activity.loc[
    lambda x: x.filter(dates).sum(axis=1) > 0,
    "trip_id",
]
trip_ids

In [ ]:
feed.trips

In [ ]:
dates = feed.get_first_week()[:1] + ["20010101"]

path = DATA_DIR / "cairns_screen_lines.geojson"
screen_lines = gpd.read_file(path)
f = feed.compute_screen_line_counts(screen_lines, dates)
f

In [ ]:
path = DATA_DIR/'cairns_gtfs.zip'
gk.list_feed(path)



In [ ]:
feed = (
    gk.read_feed(path, dist_units='km')
    .append_dist_to_stop_times()
)
feed.describe()

In [ ]:
shapes = feed.geometrize_shapes(use_utm=True)
shapes.crs
screen_lines = screen_lines.to_crs(shapes.crs)
screen_lines

In [ ]:
url = "http://transitfeeds.com/p/10-15-transit/936/latest/download"
gk.read_feed(url, dist_units="km").describe()

In [ ]:
(
    g1.crossing_time.equals(g2.crossing_time),
    g1.crossing_direction.equals(g2.orientation)
)

In [ ]:
feed.map_routes(feed.routes.route_id.iloc[:4], include_stops=False)
